<a href="https://colab.research.google.com/github/kapardhi03/100X-Components/blob/main/TalkingDocs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Multi-document summarization using Llama2 (Clustering + Graph RAG) 🦙

In [ ]:
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117 --upgrade
!pip install -q langchain einops accelerate transformers scipy
!pip install -q xformers sentencepiece sentence-transformers pypdf
!pip install -q llama-index==0.7.21 llama_hub==0.0.19 openai
!pip install -i https://test.pypi.org/simple/ bitsandbytes
!pip install glob2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.1/645.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 13.5 MB/s e

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, GenerationConfig
import huggingface_hub
from langchain import HuggingFacePipeline

huggingface_hub.notebook_login()

#Downloading the model and Text-generatioin pipeline

In [ ]:
def load_model(device_type, model_id):

    print(f"Loading Model: {model_id}, on: {device_type}")

    tokenizer = AutoTokenizer.from_pretrained(
        model_id, cache_dir="./model/"
        )

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        cache_dir="./model/",
        torch_dtype=torch.float16,
        trust_remote_code=True,
        load_in_8bit=True,
        )

    generation_config = GenerationConfig.from_pretrained(model_id)

    # Create a pipeline for text generation
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=4096,
        top_p=0.95,
        repetition_penalty=1.15,
        generation_config=generation_config,
    )

    local_llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature': 0})
    print("Local LLM Loaded")

    return tokenizer, local_llm

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
SHOW_SOURCES = True

In [ ]:
tokenizer, LLM = load_model(device, "meta-llama/Llama-2-7b-chat-hf") #Huggingface model id

Loading Model: meta-llama/Llama-2-7b-chat-hf, on: cuda


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Local LLM Loaded


#Dealing with the documents (Text extraction + clustering + adjusting chunk tokens)

In [ ]:
import os
from glob import glob
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

In [ ]:
def load_documents_and_chunks(directory): #Takes the docs directory and returns documents

  documents = []
  for item_path in glob(directory + "*.pdf"):
      loader = PyPDFLoader(item_path)
      documents.extend(loader.load())

  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size = 1024,
      chunk_overlap  = 128,  # similar token len in overlap of text between chunks
      is_separator_regex = False,
  )

  raw_text = ""
  text_chunks = []
  for doc in documents:
    raw_text += doc.page_content
    text_chunks.append(doc.page_content)

  return text_chunks, raw_text

def calc_tokens(text):
  return len(tokenizer.tokenize(text))

In [ ]:
chunks, raw_text = load_documents_and_chunks("/content/documents/")

In [ ]:
for i in chunks:
  print(calc_tokens(i))

1494
1602
1287
1530
902
1307
1354
1895
655
1078
1347
1436
1350
1301
2646
1416
1248


In [ ]:
len(chunks)

17

In [ ]:
chunks[10]

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2') #Embedding model

def cluster_sentences(sentences, distance_threshold=1.3): # AgglomerativeClustering for vector clustering
    sentence_embeddings = model.encode(sentences)

    clustering_model = AgglomerativeClustering(distance_threshold=distance_threshold, n_clusters=None, linkage='ward')
    clustering_model.fit(sentence_embeddings)

    clustered_sentences = {}
    for sentence_id, cluster_id in enumerate(clustering_model.labels_):
        if cluster_id not in clustered_sentences:
            clustered_sentences[cluster_id] = []
        clustered_sentences[cluster_id].append(sentences[sentence_id])

    return [cluster for cluster in clustered_sentences.values()]

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
clusters = cluster_sentences(chunks)

In [ ]:
for i in clusters:
  if isinstance(i, list):
    print("List length:", len(i))
  else:
    print("String")

List length: 9
List length: 2
List length: 2
List length: 4



In a cluster where `n` chunks exist, each containing a certain number of tokens, the combined token size of the cluster is represented as `(tokens[0] + tokens[1] + ... + tokens[n])`. Occasionally, this total surpasses the input token limit of the Large Language Model (LLM), making it challenging for digestion. To address this, the clusters are redistributed, imposing a token limit for each cluster. The advantage of having similar information in each cluster ensures that the choice of elements within a cluster is inconsequential, as they consistently exhibit similarity.

✅ Therefore, achieving clusters of similar information which can fit into the LLM


In [ ]:
import math
import numpy as np

def redistribution(listoflistofchunks):
  limit = 2048
  redistributed = []
  for index, chunks in enumerate(listoflistofchunks):
    tokens = calc_tokens("\n".join(chunks))

    print(index, tokens)

    if tokens < limit:
      redistributed.extend([chunks])
    else:
      print(f"chunk {index} of tokens {tokens} is splitted into {math.ceil(tokens/limit)} parts")
      partitionlist = np.array_split(chunks, math.ceil(tokens/limit))
      # res = [list(x) for x in partitionlist]
      redistributed.extend([list(x) for x in partitionlist])
  return redistributed

In [ ]:
limit_clusters = redistribution(clusters)

0 12335
chunk 0 of tokens 12335 is splitted into 7 parts
1 2250
chunk 1 of tokens 2250 is splitted into 2 parts
2 2662
chunk 2 of tokens 2662 is splitted into 2 parts
3 6614
chunk 3 of tokens 6614 is splitted into 4 parts


In [ ]:
for i in limit_clusters:
  if isinstance(i, list):
    print("List length:", len(i), "tokens:", calc_tokens("\n".join(i)))
  else:
    print("String")

List length: 2 tokens: 3097
List length: 2 tokens: 2818
List length: 1 tokens: 1895
List length: 1 tokens: 655
List length: 1 tokens: 1078
List length: 1 tokens: 1436
List length: 1 tokens: 1350
List length: 1 tokens: 902
List length: 1 tokens: 1347
List length: 1 tokens: 1307
List length: 1 tokens: 1354
List length: 1 tokens: 1301
List length: 1 tokens: 2646
List length: 1 tokens: 1416
List length: 1 tokens: 1248


In [ ]:
#joining the chunks into the cluster to make a string of all the cluster information
#Therefore it can be used by langchain with prompt templates

tosendlangchain = ["\n".join(i) for i in limit_clusters]

for i in tosendlangchain:
  print(calc_tokens(i))

3097
2818
1895
655
1078
1436
1350
902
1347
1307
1354
1301
2646
1416
1248


#Utilizing langchain for creating LLM chain and prompt templates

In [ ]:
from langchain import PromptTemplate, LLMChain

def cluster_summaries(text_chunks_list):
  template = """As an expert summarizer, produce a concise yet comprehensive summary of the given text,
  whether it's an article, blog post, conversation, or passage without adding your interpretations.
  Your summary should exhibit great detail, depth, and complexity while ensuring clarity and conciseness.
  Employ the following content to create the summary:
  {text}"""
  prompt = PromptTemplate(template=template, input_variables=["text"])
  llm_chain = LLMChain(prompt=prompt, llm=LLM)

  ip = [{'text': i} for i in text_chunks_list]
  summary = llm_chain.apply(ip)

  return [i['text'] for i in summary]


def generate_final_summary(text_chunk): #Takes the final text and generates the final summary
  template = """I want you to act as a text summarizer to help me create a brief understandable summary of the text I provide,
  whether it's an article, blog post, conversation, or passage.
  The summary should be in good length, expressing the points and concepts written in the original text without adding your interpretations.
  Breifly summarize the following text for me:
  {text}"""

  prompt = PromptTemplate(template=template, input_variables=["text"])
  llm_chain = LLMChain(prompt=prompt, llm=LLM)

  summary = llm_chain.run(text_chunk)

  return summary

In [ ]:
def recursive_summary(listofclusters):
  tokens = calc_tokens("\n".join(listofclusters))
  print("current tokens:", tokens)

  if tokens+90 <= 4096:
    return generate_final_summary("\n".join(listofclusters)) #proceed to the final summary token limit is not exceeded
  else:
    summaryofclusters = cluster_summaries(listofclusters) #else use mapreduce like technique
    print(summaryofclusters,"\n\n\n")
    return recursive_summary(summaryofclusters)

In [ ]:
summary = recursive_summary(tosendlangchain)

current tokens: 23864
['• compression: The ability to compress the sum- \nmary to a small fraction of the original document set. \n• scalability: The ability to handle large numbers of \ndocuments efficiently. \n• robustness: The ability to handle noisy or corrupt \ndocuments effectively. \n• flexibility: The ability to handle varying degrees of \ntopical overlap between documents. \n• interpretable: The ability to provide interpreta- \nble and coherent summaries. \n4 Methodology \nWe propose a methodology that combines several tech- \nniques to generate a summary of a group of documents. \nThe proposed methodology consists of three stages: \npreparation, selection, and aggregation. \nIn the preparation stage, we use domain-independ- \nent techniques to extract relevant information from each \ndocument in the group. We use a combination of statis- \ntical techniques and natural language processing to iden- \ntify important phrases, sentences, and paragraphs. We \nalso use a technique c

In [ ]:
summary

''